## Scrapping


In [1]:
from bs4 import BeautifulSoup

html_doc = 'fafea'

soup = BeautifulSoup(html_doc, 'html.parser')

In [2]:
import re
import requests
from bs4 import BeautifulSoup, Comment

In [3]:
url = f'https://www.marmiton.org/recettes/recette_gratin-pomme-de-terre-et-panais_532213.aspx' # valuepage={page}

In [4]:
response = requests.get(url, headers={"Accept-Language":"en-US"})
soup = BeautifulSoup(response.content, "html.parser")

# print(soup)

In [5]:
# from bs4 import Comment

# for child in soup.body.div.children:
#     if isinstance(child,Comment):
#         child.extract()

In [6]:
ingredients_list = []
for ingredients in soup.find_all("div", class_="fLWRho"):
    title = ingredients.find_all("span", class_="epviYI") #.find("a").string
#     duration = int(movie.find(class_="runtime").string.strip(' min'))
#     year = int(re.search(r"\d{4}", movie.find(class_="lister-item-year").string).group(0))
    ingredients_list.append(title)
#     movies.append({'title': title, 'duration': duration, 'year': year})
print(ingredients_list[1][0])
# ingredients_list[1].content
soup2 = BeautifulSoup(str(ingredients_list[1][0]), "html.parser")


# from bs4 import Comment

# if isinstance(soup2.string,Comment):
#     souped2 = soup2.extract()
#     print(souped2)
num=0
for element in soup2(text=lambda text: isinstance(text, Comment)):
    print(element.extract())

    

<span class="SHRD__sc-10plygc-0 epviYI">1<!-- --> <!-- -->branche</span>
 
 


In [7]:
print(str(soup2))

<span class="SHRD__sc-10plygc-0 epviYI">1 branche</span>


In [8]:
soupstr=str(soup2)
result=re.sub(r'<!.*?->','', soupstr)


In [9]:
BeautifulSoup(soup2.string)

TypeError: object of type 'NoneType' has no len()

In [10]:
units = ['branche', 'bouquet','cl','cuillère à café bien bombée', 'c.à.c', 'c.à.s' 'g', \
         'gousse', 'gousses', 'ml', 'plaques', 'pincée', 'poignée', 'pot', 'tablette', 'tranches', 'verre', 'verres']
doubts = ['1⁄2', '1⁄4']

In [11]:
# 1⁄2 bouquet
# de persil

## Prep nettoyage de données

In [44]:
import pandas as pd
ingredients_df = pd.read_csv('../createur_de_recette/data/ingredients.csv')
recipes_df = pd.read_csv('../createur_de_recette/data/recipes.csv')
recipes_df.head(2)

,recipe_id,recipe_url,recipe_title,recipe_type,recipe_steps
0,10534,https://www.marmiton.org/recettes/recette_le-c...,Le crumble aux pommes du Chat qui Tousse,Dessert,Préchauffer le four à 210°C (thermostat 7). \n...
1,10531,https://www.marmiton.org/recettes/recette_esca...,Escalopes lucullus à la cocotte,Plat principal,"Coupez l'oignon en petits morceaux, faites rev..."


In [13]:
recipes_df.shape

(70603, 5)

In [14]:
recipes_df['first_letter'] = recipes_df.recipe_steps.apply(lambda x : x[0])

In [15]:
recipes_df.first_letter.value_counts().sort_values()[0:49]

à       1
:       1
`       1
¨       1
'       1
➢       1
\n      2
u       2
8       2
0       2
[       2
t       2
~       2
_       2
°       3
n       3
5       3
6       3
h       4
o       4
b       4
a       5
>       5
v       5
Z       6
•       6
e       6
s       7
"       7
3       7
r       8
<      11
4      13
.      16
À      17
é      24
l      24
       32
(      32
m      38
d      44
2      44
Q      50
*      62
c      63
p      70
f      71
Ô      72
J     100
Name: first_letter, dtype: int64

In [16]:
# recipes_df.first_letter.value_counts().sort_values()[0:49]
recipes_df.first_letter.value_counts().sort_values()[46:80]

f       71
Ô       72
J      100
U      148
G      202
I      244
O      345
N      688
H      790
-      803
1      843
V      915
T      930
A     1074
S     1158
B     1342
R     1488
É     2166
L     3672
E     4434
M     6429
D     7320
C     7661
F    10659
P    16401
Name: first_letter, dtype: int64

In [17]:
DELETE_ROW_IF_FIRST_CHAR=[':','`', '¨', '➢']
REPLACE_IF_FIRST_CHAR = [' ', '-', '\n']
STR_TO_REPLACE_BY_SINGLE_OCCURENCES = ['!', '.', ' ', '-', ]



In [18]:
recipes_df.shape


(70603, 6)

In [42]:
def recipe_df_cleaning(recipes_df) :
    """Function dedicated to the cleaning of the recipe's DataFrame cooking steps
    
    Dropped :
        - steps with HTML Tags in it '<' 
        - steps starting with ':','`', '¨', '➢', '\n' ( only 1 encounters as first character in steps)
        
    Replaced characters :
        - if first char is ' ' or '-'
        
        - multiples !!! ... and spaces '    '        

    """
    # Creating a column with the first letters of each recipe_steps to identify troublemakers
    recipes_df['first_letter'] = recipes_df.recipe_steps.apply(lambda x : x[0])
    
    # Deleting HTML Tags (balises)
    rows_to_drop = list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1].index)
    recipes_df.drop(rows_to_drop, axis=0, inplace=True)
    
    recipes_df = recipes_df.reset_index().drop(columns='index')
    
    # Deleting rows with first_letter in DELETE_ROW_IF_FIRST_CHAR
    for deleted_char in DELETE_ROW_IF_FIRST_CHAR :
        rows_to_drop = list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : \
                                x.startswith(deleted_char))].index)
        
    recipes_df.drop(rows_to_drop, axis=0, inplace=True)
    recipes_df = recipes_df.reset_index().drop(columns='index')
    
    # Replacing first chars of rows if it starts with a char from REPLACE_IF_FIRST_CHAR (Twice, just in cases '- ' or ' .')
    for i in range(2) :
        for replaced_char in REPLACE_IF_FIRST_CHAR :
            recipes_df.recipe_steps = recipes_df.recipe_steps.apply(lambda x : x[1:] \
                    if x.startswith(replaced_char) else x)
            
            
    # Replacing multiples occurences like !!!!! to ! or ... to . or '        ' to ' '
    for i in range(5) : 
        for char in STR_TO_REPLACE_BY_SINGLE_OCCURENCES:
            recipes_df.recipe_steps = recipes_df.recipe_steps.apply(lambda x : x.replace(f"{char}{char}", f"{char}"))
        
    
    # dropping useless columns
    recipes_df.drop(columns='first_letter', inplace=True)
    
    return recipes_df

def drop_rows(df, rows_to_drop, index ) : 
    """ delete rows and reset index
    df : DataFrame
    rows_to_drop : list
    
    """
    df.drop(rows_to_drop, axis=0, inplace=True)
    df = df.reset_index().drop(columns='index')
    return df

trans_recipe = recipe_df_cleaning(recipes_df)
trans_recipe

,recipe_id,recipe_url,recipe_title,recipe_type,recipe_steps
0,10534,https://www.marmiton.org/recettes/recette_le-c...,Le crumble aux pommes du Chat qui Tousse,Dessert,Préchauffer le four à 210°C (thermostat 7). \n...
1,10531,https://www.marmiton.org/recettes/recette_esca...,Escalopes lucullus à la cocotte,Plat principal,"Coupez l'oignon en petits morceaux, faites rev..."
2,10529,https://www.marmiton.org/recettes/recette_terr...,Terrine de foie de porc,Plat principal,"hacher (pas trop fin) le foie, le lard, les éc..."
3,10530,https://www.marmiton.org/recettes/recette_lapi...,Lapin au vin blanc (à la cocotte),Plat principal,Faire revenir les échalotes et les lardons dan...
4,10528,https://www.marmiton.org/recettes/recette_truf...,Truffes en chocolat,Confiserie,La veille : faire fondre le chocolat au bain-m...
...,...,...,...,...,...
70396,532439,https://www.marmiton.org/recettes/recette_buch...,Bûche de Noël au chocolat et crêpes dentelles,Dessert,Faire fondre les chocolats au bain marie avec ...
70397,532434,https://www.marmiton.org/recettes/recette_cook...,Cookies aux noix de pécan et au chocolat,Dessert,Pour le beurre de pécan : Faire fondre le beur...
70398,532401,https://www.marmiton.org/recettes/recette_sand...,Sando au poulet,Plat principal,Couper le poulet en morceaux de 5/6cm. \n\n Ba...
70399,532213,https://www.marmiton.org/recettes/recette_grat...,Gratin pomme de terre et panais,Accompagnement,Éplucher les pommes de terre et le panais et l...


In [20]:
stringed = 'twto'
stringed.startswith('t')
70401 - trans_recipe.recipe_steps[trans_recipe.recipe_steps.apply(lambda x : x.find('.')) != -1].count()

307

In [39]:
recipes_df.recipe_steps[recipes_df.first_letter == '.']
# recipes_df.recipe_steps[recipes_df.first_letter == '.'].reset_index().recipe_steps[0]

AttributeError: 'DataFrame' object has no attribute 'first_letter'

In [41]:
recipes_df[recipes_df.recipe_steps.apply(lambda x : x.find("<"))!= -1].count()

recipe_id       200
recipe_url      200
recipe_title    200
recipe_type     200
recipe_steps    200
dtype: int64

In [46]:
recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1]

793      <I>Risotto ai funghi porcini</I> \n\n Commence...
856      Couper le thon en petits morceaux, ainsi que l...
1129     La veille : Piquer les parties rouge foncé aut...
1336     Préparer le thon en tranches très fines, comme...
2274     <IMG SRC="http://www.marmiton.org/pix/recettes...
                               ...                        
56168    1 - Même préparation que pour une <a href="htt...
56211    Préparer le <a href="http://www.marmiton.org/p...
58228    Faire revenir l'oignon et l'ail (le tout haché...
62690    Commencez par préparez les boulettes de <a hre...
63808    <préchauffer le four à 200° C (Thermostat 6-7)...
Name: recipe_steps, Length: 200, dtype: object

In [32]:
list(recipes_df.recipe_steps[recipes_df.recipe_steps.apply(lambda x : x.find('<')) != -1])[2]

'La veille : Piquer les parties rouge foncé autour de l’arête avec un clou de girofle par quartier (la tranche de thon se présente en 4 quartiers autour de l’arête). Piquer la chair rose d’éclats d’ail, comme on le ferait pour un gigot. \n\n Éplucher et couper les oignons en rondelles fines. \n\n Sur les 2 faces, huiler les tranches de thon, les saler, poivrer et saupoudrer légèrement de poivre de Cayenne. \n\n Les poser sur un lit d’oignons en rondelles sur lequel on aura effeuillé une branche de thym. Laisser reposer 24 heures. Retourner les tranches au moins une fois pendant cette période, pour que l’autre face s’imprègne des parfums. \n\n Le jour même, 10 à 15 min avant de déguster ces tranches de thon, commencer la cuisson au barbecue ou au grill (de préférence sans les oignons qui brûlent facilement). \n\n Ne pas trop prolonger la cuisson, car la chair deviendrait trop sèche. Ne pas trop tenir compte des parties rouge foncé autour de l’arête car on ne les mange pas en raison de l

In [24]:
r

NameError: name 'r' is not defined